## Modifying HiHydroSoil .tif Files
Modifying the files to create a series of tiles at three degrees

### Loading packages

In [3]:
from osgeo import gdal, ogr

### Set paths

In [5]:
lst_depth = ['0-5cm', '5-15cm', '15-30cm', '30-60cm', '60-100cm', '100-200cm', '200-2000cm']

output_directory = f'/home/geofelpave/Documents/6_Test_Area/data/New_Soil_Properties/Ksat_{lst_depth[5]}/'
input_raster = f'/home/geofelpave/Documents/6_Test_Area/data/Soil_Properties/Ksat/Ksat_{lst_depth[5]}_M_250m.tif'
input_vector = f'/home/geofelpave/Documents/6_Test_Area/data/Misc/fishnet_hihydrosoil_land.shp'
file_prefix = f'Ksat_{lst_depth[5]}_M_250m_'

### Working with the files to create the tiles

In [6]:

#Read shapefile in gdal
ds_vector = gdal.OpenEx(input_vector)
# Load shapefile to be able to read attributes
layer = ds_vector.GetLayer()

# Create list of grid_ref values
grid_lst = []
list_field = ['grid_ref']
for feature in layer:
    values_list = [feature.GetField(j) for j in list_field]
    grid_lst.append(values_list[0])

# #Use the list of grid to loop and clip raster
# file_prefix = 'N_5-15cm_M_250m_'
# # grid_lst = ['76E5N', '76E2N']

# Loop and with cutlineSQL select the grid to clip the raster
for i, grid_ref in enumerate(grid_lst):

    result= gdal.Warp(
        output_directory + file_prefix + f"{grid_ref}.tif",
        input_raster,
        cutlineDSName=input_vector,
        format = 'GTiff',
        srcNodata = -9999,
        dstNodata=-9999,
        cutlineSQL=f"SELECT * FROM fishnet_hihydrosoil_land WHERE grid_ref = '{grid_ref}'",
        cutlineLayer="cutline",
        cropToCutline=True,
        warpOptions=["CUTLINE_ALL_TOUCHED=True"], # this makes sure the clip is to the border of the each grid
    )

    
print(f'All done go and check {output_directory}')

All done go and check /home/geofelpave/Documents/6_Test_Area/data/New_Soil_Properties/logKferr_200-2000cm/
